In [1]:
'''
This is the methods notebook for AstrinWT-p.Q1012* Datasets.

Copyright (C) 2021 - Trupti Gore.

License: GPL-3. 
'''



'\nThis is the methods notebook for AstrinWT-p.Q1012* Datasets.\n\nCopyright (C) 2021 - Trupti Gore.\n\nLicense: GPL-3. \n'

In [2]:
# import necessary modules
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import skimage.filters as filters
import skimage.color as color
from PIL import Image
import pandas as pd
from skimage.util import img_as_float
from skimage.color import label2rgb, rgb2gray,gray2rgb
from skimage import segmentation,morphology,feature,measure,filters
from scipy import ndimage as ndi
import glob
from natsort import natsorted 
import os
import glob
import re
import random

In [21]:
# import function file
%run functions_AstrinWT.ipynb
random.seed(3)

In [16]:
# path to the local folder for the notebooks
BASE_DIR="../../07-ImageAnalysis/" 
#path to the image folder

FOLDER="set6"
IMAGE_DIR=BASE_DIR + "data/AMovie/" + FOLDER + "/"
CSV_DIR = BASE_DIR + "Kinetochore-Particle-Tracker/AnalysisResults/AMovie/" + FOLDER +"/" + "csvs/"
GRAPHS_DIR= BASE_DIR + "Kinetochore-Particle-Tracker/GraphAnalysis/AstrinWT/"



#### A List for the Images Folders

In [18]:
wt=[]
mut=[]
wt_path = IMAGE_DIR + 'wt'
mut_path=IMAGE_DIR + 'mutant'

for folder in glob.glob(wt_path + '/*'):
    wt.append(wt_path + '/' + folder.split('/')[-1].split('.')[0])
for folder in glob.glob(mut_path + '/*'):
    mut.append(mut_path + '/' + folder.split('/')[-1].split('.')[0])     


In [30]:
mut

[]

In [22]:
# For wt

for i in range(len(wt)):
    
    df_YFP_astrin,df_CENP_B_ds_red=method_cenpb(wt[i],FOLDER)#,YFP_astrin_cyto,CENP_B_ds_red_cyto,YFP_astrin_bg,CENP_B_ds_red_bg
    
    df=pd.merge(df_YFP_astrin, df_CENP_B_ds_red,  how='outer',on=['label','area']
                                                       ,suffixes=('_YFP_astrin', '_CENP_B_ds_red'))
    
    directory=wt[i].split('/')[-1].split('.')[0] # to create a folder per experiment to save csvs
    path = CSV_DIR + 'wt/' + directory
    try:
        os.makedirs(path)
    except FileExistsError:
    # directory already exists
        pass
    #pd.concat([df1,df_cdki,df_dmso],ignore_index=True)
    check_nan_in_df = df.isnull().values.any()
    if check_nan_in_df >0:
        df=df.dropna()
        print("NaNs removed from %s " %( directory))
   
    df.to_csv(path + '/' + 'particles'+'.csv') 
   

/var/folders/nw/b5trlz_d7819ystdc2gmcwbw0000gn/T/ipykernel_4660/4156651925.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_cenpb_final=pd.concat([df_cenpb_final,df_cenpb],ignore_index=True)
/var/folders/nw/b5trlz_d7819ystdc2gmcwbw0000gn/T/ipykernel_4660/4156651925.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_astrin_final=pd.concat([df_astrin_final,df_astrin],ignore_index=True)


NaNs removed from Exp2018_032_setVI_HeLa_YFP-AstrinWT_CENPBdsRed_GFP_mcherry_01_R3D_D3D 


/var/folders/nw/b5trlz_d7819ystdc2gmcwbw0000gn/T/ipykernel_4660/4156651925.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_cenpb_final=pd.concat([df_cenpb_final,df_cenpb],ignore_index=True)
/var/folders/nw/b5trlz_d7819ystdc2gmcwbw0000gn/T/ipykernel_4660/4156651925.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_astrin_final=pd.concat([df_astrin_final,df_astrin],ignore_index=True)
/var/folders/nw/b5trlz_d7819ystdc2gmcwbw0000gn/T/ipykernel_4660/4156651925.py:64: FutureWarning: The 

<Figure size 640x480 with 0 Axes>

In [29]:
# For control

for i in range(len(mut)):
    
    df_YFP_astrin,df_CENP_B_ds_red=method_cenpb(mut[i],FOLDER) #,YFP_astrin_cyto,CENP_B_ds_red_cyto,YFP_astrin_bg,CENP_B_ds_red_bg
    
    df=pd.merge(df_YFP_astrin, df_CENP_B_ds_red,  how='outer',on=['label','area']
                                                       ,suffixes=('_YFP_astrin', '_CENP_B_ds_red'))
    
    directory=mut[i].split('/')[-1].split('.')[0] # to create a folder per experiment to save csvs
    path = CSV_DIR + 'mut/' + directory
    try:
        os.makedirs(path)
    except FileExistsError:
    # directory already exists
        pass
    #pd.concat([df1,df_cdki,df_dmso],ignore_index=True)
    check_nan_in_df = df.isnull().values.any()
    if check_nan_in_df >0:
        df=df.dropna()
        print("NaNs removed from %s " %( directory))
    df.to_csv(path + '/' + 'particles'+'.csv') 
   

### Generate csv files for all cells

In [25]:

# The paths are for the generated csv files
wt_csv=[]
mut_csv=[]
wt_path = CSV_DIR + 'wt'
mut_path=CSV_DIR + 'mut'

for folder in natsorted(glob.glob(wt_path + '/*')):
    wt_csv.append(wt_path + '/' + folder.split('/')[-1].split('.')[0])
    
for folder in natsorted(glob.glob(mut_path + '/*')):
    mut_csv.append(mut_path + '/' + folder.split('/')[-1].split('.')[0])  
  

In [26]:
# read wildtype csvs to a dataframe
cell_count=1
path = GRAPHS_DIR + FOLDER
try:
    os.makedirs(path)
except FileExistsError:
    # directory already exists
    pass
df_graph_wt=pd.DataFrame()
for i in range(len(wt_csv)):
    df_wt=pd.read_csv(wt_csv[i]+'/'+'Particles.csv')
    df_wt['CENP_B_ds_red@kt/CENP_B_ds_red@cyto']= df_wt['mean_intensity_CENP_B_ds_red']/df_wt['CENP_B_ds_red_cyto']
    df_wt['YFP_astrin@kt/YFP_astrin@cyto']= df_wt['mean_intensity_YFP_astrin']/df_wt['YFP_astrin_cyto']
    df_wt['CENP_B_ds_red@kt/CENP_B_ds_red@cyto:YFP_astrin@kt/YFP_astrin@cyto']= df_wt['CENP_B_ds_red@kt/CENP_B_ds_red@cyto'] / df_wt['YFP_astrin@kt/YFP_astrin@cyto']
    df_wt['condition']='WildType'
    df_wt['cell']=FOLDER+"_"+'WildT_cell_'+str(cell_count)
    expt_name=re.sub(r'^.+/([^/]+)$', r'\1', wt_csv[i])
    df_wt['Experiment']= expt_name
    fname=FOLDER+'wt_'+ str(i)
    df_wt_final=outliers_astrin(df_wt)
    df_wt_final.to_csv(GRAPHS_DIR + FOLDER+'/' + fname +'.csv') 
    df_graph_wt=pd.concat([df_graph_wt,df_wt_final],ignore_index=True)    
    cell_count+=1
df_graph_wt = df_graph_wt.drop(columns=["Unnamed: 0"], axis=1)    

df_graph_wt.to_csv(GRAPHS_DIR + FOLDER+'/' +FOLDER+ "_" +'wt_astrin_graphs'+'.csv') 



In [27]:

# read q1012
cell_count=1
path = GRAPHS_DIR + FOLDER
try:
    os.makedirs(path)
except FileExistsError:
    # directory already exists
    pass
df_graph_mut=pd.DataFrame()
for i in range(len(mut_csv)):
    df_mut=pd.read_csv(mut_csv[i]+'/'+'Particles.csv')
    df_mut['CENP_B_ds_red@kt/CENP_B_ds_red@cyto']= df_mut['mean_intensity_CENP_B_ds_red']/df_mut['CENP_B_ds_red_cyto']
    df_mut['YFP_astrin@kt/YFP_astrin@cyto']= df_mut['mean_intensity_YFP_astrin']/df_mut['YFP_astrin_cyto']
    df_mut['CENP_B_ds_red@kt/CENP_B_ds_red@cyto:YFP_astrin@kt/YFP_astrin@cyto']= df_mut['CENP_B_ds_red@kt/CENP_B_ds_red@cyto'] / df_mut['YFP_astrin@kt/YFP_astrin@cyto']
    df_mut['condition']='Q1012'
    df_mut['cell']=FOLDER+"_"+'Q1012_cell_'+str(cell_count)
    expt_name=re.sub(r'^.+/([^/]+)$', r'\1', mut_csv[i])
    df_mut['Experiment']= expt_name
    fname=FOLDER+'mut_'+str(i)
    df_mut_final=outliers_astrin(df_mut)
    df_mut_final.to_csv(GRAPHS_DIR + FOLDER+'/' + fname +'.csv') 
    df_graph_mut=pd.concat([df_graph_mut,df_mut_final],ignore_index=True)  
    cell_count+=1
    
df_graph_mut = df_graph_mut.drop(columns=["Unnamed: 0"], axis=1)    

df_graph_mut.to_csv(GRAPHS_DIR + FOLDER+'/' +FOLDER+ "_" + 'mut_astrin_graphs'+'.csv') 


In [28]:
df_graph=pd.DataFrame()
df_graph=pd.concat([df_graph,df_graph_wt,df_graph_mut],ignore_index=True)
print(len(df_graph_wt))
print(len(df_graph_mut))
print(len(df_graph))
df_graph.head()
df_graph.to_csv(GRAPHS_DIR + FOLDER+'/' +FOLDER+ "_" + 'astrin_wt_combined_graphs'+'.csv') 
#df_graph.head()

984
1107
2091
